In [2]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import Imputer
from sklearn import preprocessing


In [254]:
data = pd.read_pickle('objects/dataForCatBoost')

In [255]:
data.iloc[:,1:] = data.iloc[:,1:].replace(365243, np.NaN)
data.iloc[:,1:] = data.iloc[:,1:].replace('XAP', np.NaN)
data.iloc[:,1:] = data.iloc[:,1:].replace('XNA', np.NaN)
#data.isnull().sum()

In [256]:
#Function to capture cetegorical variables 
def _get_categorical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype=='object']
    return feats 
catFeatures=_get_categorical_features(data)
#catFeatures 

In [257]:
#Function to capture numerical variables 
def _get_numerical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype!='object']
    return feats 
numFeatures=_get_numerical_features(data)
numFeatures=numFeatures[1:]

In [258]:
catInd=np.where(data.dtypes==object)[0]
numInd= np.where(data.dtypes!=object)[0] 
numInd=numInd[1:]

In [259]:
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 
data=_factorize_categoricals(data, catFeatures)

In [260]:
data=data.iloc[:,:].values
#data[:,(catInd)]

In [261]:
numImputer = Imputer(missing_values ='NaN', strategy='median', axis=0)
numImputer= numImputer.fit(data[:,(numInd)])
data[:,(numInd)]=numImputer.transform(data[:,(numInd)])

In [262]:
catImputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis =0)
catImputer=catImputer.fit(data[:,(catInd)])
data[:,(catInd)]=catImputer.transform(data[:,(catInd)])

In [263]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data[:,(numInd)] = min_max_scaler.fit_transform(data[:,(numInd)])

In [265]:
data1 = pd.read_pickle('objects/dataForCatBoost')
data=pd.DataFrame(data=data[0:,0:],    # values
             #index=data[0:,0],    # 1st column as index
             columns=data1.columns.values) 

In [266]:
def convert(df,catFeatures):
    for col in catFeatures:
        df[col]=df[col].astype(str)
    return df
data=convert(data,catFeatures)

In [267]:
data['SK_ID_CURR']=data['SK_ID_CURR'].astype(int)
data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CN_CNT_DRAWINGS_POS_CURRENT,CN_CNT_INSTALMENT_MATURE_CUM,CN_SK_DPD,CN_SK_DPD_DEF,CC_NAME_CONTRACT_STATUS,PCN_MONTHS_BALANCE,PCN_CNT_INSTALMENT,PCN_CNT_INSTALMENT_FUTURE,PCN_SK_DPD,PCN_SK_DPD_DEF
0,100002,0.0,0.0,0.0,0.0,0.0,0.001512,0.090287,0.090032,0.077441,...,0.00044,0.05201,0.0,0.0,-1.0,0.905263,0.323944,0.250000,0.0,0.0
1,100003,0.0,1.0,0.0,1.0,0.0,0.002089,0.311736,0.132924,0.271605,...,0.00044,0.05201,0.0,0.0,-1.0,0.549624,0.128270,0.096429,0.0,0.0
2,100004,1.0,0.0,1.0,0.0,0.0,0.000358,0.022472,0.020025,0.023569,...,0.00044,0.05201,0.0,0.0,-1.0,0.742105,0.038732,0.037500,0.0,0.0
3,100006,0.0,1.0,0.0,0.0,0.0,0.000935,0.066837,0.109477,0.063973,...,0.00044,0.00000,0.0,0.0,0.0,0.909273,0.154930,0.144167,0.0,0.0
4,100007,0.0,0.0,0.0,0.0,0.0,0.000819,0.116854,0.078975,0.117845,...,0.00044,0.05201,0.0,0.0,-1.0,0.656459,0.201878,0.149495,0.0,0.0


In [269]:
data.to_pickle('objects/dataMissingRemoved')